<a href="https://colab.research.google.com/github/anjana-psvel/Precog_Task/blob/main/word_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
import nltk
nltk.download('punkt')
!pip install nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
df= pd.read_csv("/content/brown.csv")
print(df)

      filename  para_id  sent_id  \
0         cd05        0        0   
1         cd05        0        1   
2         cd05        0        2   
3         cd05        0        3   
4         cd05        0        4   
...        ...      ...      ...   
57335     cj14        6        3   
57336     cj14        6        4   
57337     cj14        6        5   
57338     cj14        6        6   
57339     cj14        6        7   

                                                raw_text  \
0      Furthermore/rb ,/, as/cs an/at encouragement/n...   
1      The/at Unitarian/jj clergy/nns were/bed an/at ...   
2      Ezra/np Stiles/np Gannett/np ,/, an/at honorab...   
3      Even/rb so/rb ,/, Gannett/np judiciously/rb ar...   
4      We/ppss today/nr are/ber not/* entitled/vbn to...   
...                                                  ...   
57335  For/in the/at most/ap part/nn ,/, this/dt disc...   
57336                                     A/np-hl ./.-hl   
57337  Standard/jj-hl prepa

In [3]:
text = df['tokenized_text'].tolist()
len(text) # no. of sentences in the corpus


57340

In [4]:
import re

def remove_punctuation(sentence):
    # Define a regular expression pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    # Use re.sub() to replace punctuation with an empty string
    cleaned_sentence = re.sub(punctuation_pattern, '', sentence)
    return cleaned_sentence

text1=[]
for i in text:
  text1.append(remove_punctuation(i))


In [5]:
text1[1]
texts = df.tokenized_text.apply(gensim.utils.simple_preprocess)
texts

0        [furthermore, as, an, encouragement, to, revis...
1        [the, unitarian, clergy, were, an, exclusive, ...
2        [ezra, stiles, gannett, an, honorable, represe...
3        [even, so, gannett, judiciously, argued, the, ...
4        [we, today, are, not, entitled, to, excoriate,...
                               ...                        
57335    [for, the, most, part, this, discussion, will,...
57336                                                   []
57337    [standard, preparations, and, units, of, thyro...
57338    [the, international, unit, adopted, to, make, ...
57339    [the, international, unit, is, equipotent, wit...
Name: tokenized_text, Length: 57340, dtype: object

In [6]:
model = gensim.models.Word2Vec(texts,  window=5, min_count=2, workers=4)

In [7]:
#build vocabulary
model.build_vocab(texts, progress_per=1000)

In [8]:
model.train(texts, total_examples=model.corpus_count, epochs=model.epochs)

(3709395, 4904835)

In [9]:
model.wv.similarity(w1="happy", w2="inexpensive")

0.6583763

In [10]:
model.save("./word2vec-word_similarity.model")

In [11]:
from gensim.models import Word2Vec

# Load the saved Word2Vec model
model1 = Word2Vec.load("/content/word2vec-word_similarity.model")



In [12]:
df1 = pd.read_csv('/content/SimLex-999.txt', sep='\t')
df1

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [13]:
def calculate_similarity(x):
    try:
        similarity_score = model.wv.similarity(w1=x['word1'], w2=x['word2'])
        return similarity_score
    except KeyError:
        return None

# Assuming df1 is your DataFrame
df1['similarity_scores_calc'] = df1.apply(calculate_similarity, axis=1)
df1.head(10)

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),similarity_scores_calc
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41,0.230686
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67,0.864078
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19,0.660943
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18,0.749681
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93,0.839978
5,fast,rapid,A,8.75,3.32,3.07,2,5.66,1,1.68,0.589610
6,happy,glad,A,9.17,2.56,2.36,1,5.49,1,1.59,0.854110
7,short,long,A,1.23,3.61,3.18,2,5.36,1,1.58,0.715862
8,stupid,dumb,A,9.58,1.75,2.36,1,5.26,1,1.48,0.774818
9,weird,strange,A,8.93,1.59,1.86,1,4.26,1,1.30,0.796779


In [14]:
# prompt: what to replace None with?

df1['similarity_scores_calc'].fillna(0, inplace=True)
# prompt: count of "None" present in teh similarity_score column

df1['similarity_scores_calc'].isna().sum()



0

In [15]:
# prompt: give the max an dmin value of similarity_scores_calc

print("Maximum similarity score:", df1['similarity_scores_calc'].max())
print("Minimum similarity score:", df1['similarity_scores_calc'].min())


Maximum similarity score: 0.9582183957099915
Minimum similarity score: -0.07167968899011612


In [16]:
print("Maximum similarity score:", df1['SimLex999'].max())
print("Minimum similarity score:", df1['SimLex999'].min())

Maximum similarity score: 9.8
Minimum similarity score: 0.23


In [17]:
# change the range of Simlex999 to 0 to 1 and make it into a new column

df1['SimLex999_scaled'] = (df1['SimLex999'] - df1['SimLex999'].min()) / (df1['SimLex999'].max() - df1['SimLex999'].min())
df1.head()


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),similarity_scores_calc,SimLex999_scaled
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41,0.230686,0.141066
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67,0.864078,0.937304
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19,0.660943,0.892372
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18,0.749681,0.973877
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93,0.839978,0.075235


In [18]:
#scale similarity_scores_calc column to 0 to 1

df1['similarity_scores_calc_scaled'] = (df1['similarity_scores_calc'] - df1['similarity_scores_calc'].min()) / (df1['similarity_scores_calc'].max() - df1['similarity_scores_calc'].min())
df1.head()


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),similarity_scores_calc,SimLex999_scaled,similarity_scores_calc_scaled
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41,0.230686,0.141066,0.293588
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67,0.864078,0.937304,0.908592
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19,0.660943,0.892372,0.711355
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18,0.749681,0.973877,0.797517
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93,0.839978,0.075235,0.885192


In [19]:
df1

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),similarity_scores_calc,SimLex999_scaled,similarity_scores_calc_scaled
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41,0.230686,0.141066,0.293588
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67,0.864078,0.937304,0.908592
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19,0.660943,0.892372,0.711355
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18,0.749681,0.973877,0.797517
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93,0.839978,0.075235,0.885192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99,0.865369,0.273772,0.909846
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44,0.795503,0.150470,0.842009
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97,0.824496,0.477534,0.870159
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75,0.738663,0.286311,0.786818


In [20]:
# mse between similarity_scores_calc_scaled and SimLex999_scaled

from sklearn.metrics import mean_squared_error

# Calculate the mean squared error (MSE) between the two columns
mse = mean_squared_error(df1['similarity_scores_calc_scaled'], df1['SimLex999_scaled'])

# Print the MSE
print("MSE:", mse)


MSE: 0.19197523987009574


In [23]:
from sklearn.metrics import mean_absolute_error, r2_score

# Calculate the mean absolute error (MAE) between the two columns
mae = mean_absolute_error(df1['similarity_scores_calc_scaled'], df1['SimLex999_scaled'])
print("MAE:", mae)


MAE: 0.36545015648869805
